In [1]:
!pip install -q gdown torch optuna transformers[torch] transformers datasets evaluate huggingface_hub

In [2]:
!gdown --id 1OpgVTe-wbGKQ0ThbH_h4WdxAq8siNp9p

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1OpgVTe-wbGKQ0ThbH_h4WdxAq8siNp9p
To: /kaggle/working/train_berita_resampled2.csv
100%|██████████████████████████████████████| 2.18M/2.18M [00:00<00:00, 97.0MB/s]


In [3]:
!gdown --id 1D7ZvxgaHWzqz76OFruypy018si4lpvCe

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1D7ZvxgaHWzqz76OFruypy018si4lpvCe
To: /kaggle/working/test_berita.csv
100%|███████████████████████████████████████| 1.40M/1.40M [00:00<00:00, 124MB/s]


In [4]:
import pandas as pd
df = pd.read_csv("/kaggle/working/train_berita_resampled2.csv")
df

,text,label
0,juni ingat hari keluarga nasional harganas har...,neutral
1,dewan pimpin daerah dpd pdip jawa barat siap m...,neutral
2,banyak balita kabupaten indramayu alami stunti...,neutral
3,pam jaya lalu dharma wanita pam jaya menandata...,neutral
4,usai tutup rakernas iv ketua umum pdip preside...,neutral
...,...,...
1052,ketua tkd prabowo gibran jawa barat ridwan kam...,positive
1053,ketua umum pan zulkifli hasan ikut sosialisasi...,positive
1054,bakal calon presiden prabowo subianto sorot ti...,negative
1055,bakal capres prabowo subianto bicara salah sat...,negative


In [5]:
test = pd.read_csv("/kaggle/working/test_berita.csv")
test

,text,label
0,asisten kesra setda dki jakarta uus kuswanto h...,neutral
1,gubernur bal wayan koster layang surat preside...,negative
2,perintah kabupaten pemkab bandung gelar jambor...,positive
3,menteri koordinator menko bidang ekonomi airla...,neutral
4,menteri uang sri mulyani indrawati hari lantik...,neutral
...,...,...
694,presiden joko widodo mewanti wanti kemenkes ri...,negative
695,kasus stunting balita aktif surabaya terus tur...,negative
696,lowong buat mahasiswa buka oleh badan riset in...,neutral
697,menteri koordinator bidang bangun manusia buda...,neutral


In [7]:
import pandas as pd
import numpy as np
from datasets import Dataset

In [8]:
label_dict = {'positive': 0, 'neutral': 1, 'negative': 2,}  # Sesuaikan dengan label yang Anda miliki
df['label'] = df['label'].map(label_dict).tolist()
df.head()

,text,label
0,juni ingat hari keluarga nasional harganas har...,1
1,dewan pimpin daerah dpd pdip jawa barat siap m...,1
2,banyak balita kabupaten indramayu alami stunti...,1
3,pam jaya lalu dharma wanita pam jaya menandata...,1
4,usai tutup rakernas iv ketua umum pdip preside...,1


In [9]:
df["label"].value_counts(), test["label"].value_counts()

(label
 1    370
 2    347
 0    340
 Name: count, dtype: int64,
 label
 neutral     541
 negative     90
 positive     68
 Name: count, dtype: int64)

In [10]:
label_dict = {'positive': 0, 'neutral': 1, 'negative': 2,}  # Sesuaikan dengan label yang Anda miliki
test['label'] = test['label'].map(label_dict).tolist()
test.head()

,text,label
0,asisten kesra setda dki jakarta uus kuswanto h...,1
1,gubernur bal wayan koster layang surat preside...,2
2,perintah kabupaten pemkab bandung gelar jambor...,0
3,menteri koordinator menko bidang ekonomi airla...,1
4,menteri uang sri mulyani indrawati hari lantik...,1


In [11]:
# train_data = df.sample(frac=0.8, random_state=42)
# test_data = df.drop(train_data.index)

In [12]:
X_train = Dataset.from_pandas(df)
X_test = Dataset.from_pandas(test)

In [13]:
print(X_train)
print(X_test)

Dataset({
    features: ['text', 'label'],
    num_rows: 1057
})
Dataset({
    features: ['text', 'label'],
    num_rows: 699
})


In [14]:
from transformers import BertTokenizer, BertForSequenceClassification, set_seed

max_length = 512

tokenizer = BertTokenizer.from_pretrained("ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa", max_length=max_length)


def tokenize_function(examples):

    # process the input sequence
    tokenized_input = tokenizer(examples["text"],
                                truncation=True,
                                padding='max_length',
                                max_length=max_length)
    # process the labels
    tokenized_input['label'] = examples['label']

    return tokenized_input

tokenized_train_data = X_train.map(tokenize_function, batched=True)
tokenized_test_data = X_test.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/476k [00:00<?, ?B/s]

Map:   0%|          | 0/1057 [00:00<?, ? examples/s]

Map:   0%|          | 0/699 [00:00<?, ? examples/s]

In [15]:
# define the metrics

import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    report = classification_report(labels, pred, digits=4)
    acc = accuracy_score(y_true=labels, y_pred=pred)
    rec = recall_score(y_true=labels, y_pred=pred, average="macro")
    prec = precision_score(y_true=labels, y_pred=pred, average="macro")
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    print("Classification Report:\n{}".format(report))
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

In [16]:
compute_metrics(
    [
        [[1,3,4], [1,3,4], [1,3,4], [1,3,4], [4,3,1], [1,3,4], ],
        [0, 1, 2, 0, 0, 1]
    ]
)

Classification Report:
              precision    recall  f1-score   support

           0     1.0000    0.3333    0.5000         3
           1     0.0000    0.0000    0.0000         2
           2     0.2000    1.0000    0.3333         1

    accuracy                         0.3333         6
   macro avg     0.4000    0.4444    0.2778         6
weighted avg     0.5333    0.3333    0.3056         6



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classif

{'accuracy': 0.3333333333333333,
 'precision': 0.39999999999999997,
 'recall': 0.4444444444444444,
 'f1': 0.2777777777777778}

In [17]:
tokenized_train_data

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1057
})

In [18]:
import wandb
wandb.init(mode="disabled")

In [19]:
from transformers import Trainer, TrainingArguments, set_seed
from transformers import BertTokenizer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import tensorflow as tf

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
best_accuracy = 0.0

np.random.seed(42)
tf.random.set_seed(42)
set_seed(42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(tokenized_train_data)):
    if fold != 4:
        continue
    print(f"Training fold {fold}...")
    train_dataset = tokenized_train_data.select(train_idx)
    val_dataset = tokenized_train_data.select(val_idx)
    
    model = BertForSequenceClassification.from_pretrained(
        "ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa", 
        num_labels=3
    )
    
    output_dir = f"./22best_berita_bert_model_fold_{fold+1}"
    training_args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=10,  # Sesuaikan jumlah epoch
        per_device_train_batch_size=8,  # Sesuaikan batch size
        per_device_eval_batch_size=8,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    # Evaluasi pada validation set
    metrics = trainer.evaluate()
    accuracy = metrics["eval_accuracy"]
    print(accuracy)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
#         trainer.save_model(output_dir)  # Simpan model terbaik


2024-07-17 18:40:52.157229: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 18:40:52.157358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 18:40:52.300713: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Training fold 4...


config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.817931,0.691943,0.790339,0.672659,0.645000
2,No log,0.584421,0.777251,0.784079,0.777814,0.776602
3,No log,1.096910,0.739336,0.756238,0.743861,0.737846
4,No log,0.997548,0.824645,0.824746,0.823631,0.823170
5,0.404000,1.127520,0.810427,0.810834,0.806670,0.807115
6,0.404000,1.194290,0.819905,0.818844,0.819141,0.818913
7,0.404000,1.224393,0.843602,0.847653,0.842924,0.843115
8,0.404000,1.255358,0.834123,0.837041,0.833534,0.833611
9,0.404000,1.268091,0.829384,0.831591,0.828839,0.828945
10,0.006700,1.289410,0.824645,0.826426,0.824144,0.824155


Classification Report:
              precision    recall  f1-score   support

           0     0.7848    0.8732    0.8267        71
           1     1.0000    0.2500    0.4000        64
           2     0.5862    0.8947    0.7083        76

    accuracy                         0.6919       211
   macro avg     0.7903    0.6727    0.6450       211
weighted avg     0.7785    0.6919    0.6546       211

Classification Report:
              precision    recall  f1-score   support

           0     0.8226    0.7183    0.7669        71
           1     0.6582    0.8125    0.7273        64
           2     0.8714    0.8026    0.8356        76

    accuracy                         0.7773       211
   macro avg     0.7841    0.7778    0.7766       211
weighted avg     0.7903    0.7773    0.7796       211

Classification Report:
              precision    recall  f1-score   support

           0     0.6897    0.8451    0.7595        71
           1     0.6944    0.7812    0.7353        64
      

Classification Report:
              precision    recall  f1-score   support

           0     0.7778    0.8873    0.8289        71
           1     0.8387    0.8125    0.8254        64
           2     0.9265    0.8289    0.8750        76

    accuracy                         0.8436       211
   macro avg     0.8477    0.8429    0.8431       211
weighted avg     0.8498    0.8436    0.8445       211

0.8436018957345972


In [29]:
trainer.state.log_history

[{'eval_loss': 0.7542638182640076,
  'eval_accuracy': 0.7477638640429338,
  'eval_precision': 0.5591507963684192,
  'eval_recall': 0.7477638640429338,
  'eval_f1': 0.6398470730193374,
  'eval_runtime': 9.6059,
  'eval_samples_per_second': 58.194,
  'eval_steps_per_second': 7.287,
  'epoch': 1.0,
  'step': 280},
 {'loss': 0.7777,
  'grad_norm': 3.222450017929077,
  'learning_rate': 4.107142857142857e-05,
  'epoch': 1.7857142857142856,
  'step': 500},
 {'eval_loss': 0.8219510316848755,
  'eval_accuracy': 0.7495527728085868,
  'eval_precision': 0.6961099249171266,
  'eval_recall': 0.7495527728085868,
  'eval_f1': 0.6440340064053368,
  'eval_runtime': 9.6,
  'eval_samples_per_second': 58.229,
  'eval_steps_per_second': 7.292,
  'epoch': 2.0,
  'step': 560},
 {'eval_loss': 0.7494345307350159,
  'eval_accuracy': 0.7495527728085868,
  'eval_precision': 0.6961099249171266,
  'eval_recall': 0.7495527728085868,
  'eval_f1': 0.6440340064053368,
  'eval_runtime': 9.6113,
  'eval_samples_per_second

In [23]:
metrics

{'eval_loss': 1.2243927717208862,
 'eval_accuracy': 0.8436018957345972,
 'eval_precision': 0.8476526811441424,
 'eval_recall': 0.8429237706943415,
 'eval_f1': 0.843114731272626,
 'eval_runtime': 3.6949,
 'eval_samples_per_second': 57.106,
 'eval_steps_per_second': 7.307,
 'epoch': 10.0}

In [28]:
print("lol")

lol


In [21]:
train_dataset(input_ids)

NameError: name 'input_ids' is not defined

In [14]:
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer

model = BertForSequenceClassification.from_pretrained(
    "ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa", 
    num_labels=3
)

2024-07-09 18:33:17.812359: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 18:33:17.812462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 18:33:17.937911: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [19]:
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
best_accuracy = 0.0

for fold, (train_idx, val_idx) in enumerate(kfold.split(tokenized_train_data)):
    if fold != 4:
        continue
    print(f"Training fold {fold}...")
    train_dataset = tokenized_train_data.select(train_idx)
    val_dataset = tokenized_train_data.select(val_idx)
    
    model = RobertaForSequenceClassification.from_pretrained(
        "ayameRushia/roberta-base-indonesian-sentiment-analysis-smsa", 
        num_labels=3
    )
    
    output_dir = f"./22best_berita_roberta_model_fold_{fold+1}"
    training_args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=10,  # Sesuaikan jumlah epoch
        per_device_train_batch_size=8,  # Sesuaikan batch size
        per_device_eval_batch_size=8,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Evaluasi pada validation set
    metrics = trainer.evaluate()
    accuracy = metrics["eval_accuracy"]
    print("accuracy")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
#         trainer.save_model(output_dir)  # Simpan model terbaik


2024-07-17 18:02:46.833504: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 18:02:46.833606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 18:02:46.975477: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Training fold 3...


config.json:   0%|          | 0.00/954 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.636174,0.763033,0.772081,0.754443,0.752265
2,No log,0.610353,0.791469,0.811126,0.779168,0.778419
3,No log,0.922530,0.838863,0.844121,0.839387,0.834851
4,No log,0.777563,0.853081,0.850456,0.852591,0.851307
5,0.437200,0.935689,0.853081,0.854539,0.849875,0.848158
6,0.437200,1.297493,0.834123,0.857260,0.833591,0.828914
7,0.437200,0.931626,0.857820,0.856288,0.858867,0.856451
8,0.437200,1.242460,0.819905,0.826716,0.822141,0.815853
9,0.437200,1.274166,0.838863,0.854870,0.835205,0.832372
10,0.012400,1.242619,0.834123,0.847573,0.831140,0.827818


Classification Report:
              precision    recall  f1-score   support

           0     0.6892    0.8095    0.7445        63
           1     0.8444    0.5758    0.6847        66
           2     0.7826    0.8780    0.8276        82

    accuracy                         0.7630       211
   macro avg     0.7721    0.7544    0.7523       211
weighted avg     0.7741    0.7630    0.7581       211

Classification Report:
              precision    recall  f1-score   support

           0     0.9000    0.5714    0.6990        63
           1     0.6628    0.8636    0.7500        66
           2     0.8706    0.9024    0.8862        82

    accuracy                         0.7915       211
   macro avg     0.8111    0.7792    0.7784       211
weighted avg     0.8144    0.7915    0.7877       211

Classification Report:
              precision    recall  f1-score   support

           0     0.7595    0.9524    0.8451        63
           1     0.8868    0.7121    0.7899        66
      

Classification Report:
              precision    recall  f1-score   support

           0     0.8143    0.9048    0.8571        63
           1     0.8571    0.8182    0.8372        66
           2     0.8974    0.8537    0.8750        82

    accuracy                         0.8578       211
   macro avg     0.8563    0.8589    0.8565       211
weighted avg     0.8600    0.8578    0.8578       211

accuracy


In [20]:
metrics

{'eval_loss': 0.9316259622573853,
 'eval_accuracy': 0.8578199052132701,
 'eval_precision': 0.8562881562881562,
 'eval_recall': 0.8588674198430296,
 'eval_f1': 0.8564507198228127,
 'eval_runtime': 3.841,
 'eval_samples_per_second': 54.934,
 'eval_steps_per_second': 7.029,
 'epoch': 10.0}

In [17]:
import wandb
wandb.init(mode="disabled")

In [20]:
output_dir = "stunting-medsos-sentiment" # name your own output directory
training_args = TrainingArguments(output_dir=output_dir,
                                  eval_strategy="epoch",
                                  num_train_epochs=10,
                                  push_to_hub=True,
                                 )

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.474943,0.836653,0.861473,0.727124,0.750060
2,0.449200,0.792981,0.788845,0.737806,0.751903,0.741038
3,0.449200,1.137557,0.794821,0.764995,0.790704,0.764099
4,0.161600,1.111278,0.814741,0.767578,0.763460,0.765437
5,0.161600,1.166318,0.818725,0.775874,0.779726,0.777162
6,0.038700,1.356900,0.814741,0.773195,0.776454,0.774124
7,0.038700,1.424671,0.818725,0.777714,0.766886,0.771421
8,0.008900,1.419205,0.824701,0.785764,0.757234,0.768561
9,0.008900,1.474707,0.814741,0.772430,0.769644,0.770947
10,0.003300,1.480873,0.816733,0.774366,0.770967,0.772536


Classification Report:
              precision    recall  f1-score   support

           0     0.9000    0.3418    0.4954        79
           1     0.8765    0.8713    0.8739       171
           2     0.8079    0.9683    0.8809       252

    accuracy                         0.8367       502
   macro avg     0.8615    0.7271    0.7501       502
weighted avg     0.8458    0.8367    0.8178       502

Classification Report:
              precision    recall  f1-score   support

           0     0.5422    0.5696    0.5556        79
           1     0.7685    0.9123    0.8342       171
           2     0.9028    0.7738    0.8333       252

    accuracy                         0.7888       502
   macro avg     0.7378    0.7519    0.7410       502
weighted avg     0.8003    0.7888    0.7899       502

Classification Report:
              precision    recall  f1-score   support

           0     0.4921    0.7848    0.6049        79
           1     0.9236    0.7778    0.8444       171
      

TrainOutput(global_step=2520, training_loss=0.13128438797679598, metrics={'train_runtime': 1237.3873, 'train_samples_per_second': 16.236, 'train_steps_per_second': 2.037, 'total_flos': 5285948562155520.0, 'train_loss': 0.13128438797679598, 'epoch': 10.0})

In [20]:
trainer.evaluate()

Classification Report:
              precision    recall  f1-score   support

           0     0.7529    0.6038    0.6702       106
           1     0.8802    0.8284    0.8535       204
           2     0.8547    0.9434    0.8969       318

    accuracy                         0.8487       628
   macro avg     0.8293    0.7919    0.8069       628
weighted avg     0.8458    0.8487    0.8445       628



{'eval_loss': 0.9211301207542419,
 'eval_accuracy': 0.8487261146496815,
 'eval_precision': 0.8292834548349255,
 'eval_recall': 0.7918670612899247,
 'eval_f1': 0.806851136048422,
 'eval_runtime': 10.7519,
 'eval_samples_per_second': 58.408,
 'eval_steps_per_second': 7.348,
 'epoch': 10.0}

In [21]:
# save model
trainer.save_model("./22best_berita_bert_model_fold_5")

In [22]:
# push
trainer.push_to_hub(commit_message="Training fold 5")

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

events.out.tfevents.1721241666.9b52e4c40538.34.0:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1721242210.9b52e4c40538.34.1:   0%|          | 0.00/560 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/choiruzzia/22best_berita_bert_model_fold_5/commit/adb17af49180df7a07f29baf282bd851df827285', commit_message='Training fold 5', commit_description='', oid='adb17af49180df7a07f29baf282bd851df827285', pr_url=None, pr_revision=None, pr_num=None)

In [34]:
NOMOR = 774
df.iloc[NOMOR]["text"], df.iloc[NOMOR]["label"]

('data riset sehat dasar riskesdas tahun tunjuk angka prevalensi balita indonesia capai persen arti satu tiga balita alami stunting walaupun akhir angka prevalensi balita stunting indonesia turun jadi persen indonesia sendiri hingga kini milik beban stunting besar dua asia tenggara lima dunia desember angka persen indonesia milik target ambisius angka prevalensi stunting jadi persen tahun turut dukung program perintah indonesia vasa hotel surabaya prenagen bawah naung kalbe nutritionals laku giat csr stunting puskesmas dukuh kupang surabaya kamis lebih butir telur susu uht kotak susu prenagen mommy bagi terima diri balita stunting ibu hamil kurang energi kronis kek giat csr rupa bagi vasa touch sudah sekian kali laku vasa hotel surabaya vasa touch rupa program csr gagas vasa hotel telah sekian lama vasa hotel laku program csr kala istilah baru maksud giat amal laku vasa dokumentasi informasi baik internal semua tuju publik internal vasa tahu tiap dedikasi mereka laku kerja vasa beri ma

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Bidirectional, BatchNormalization
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import time

# Hyperparameters
vocab_size = 10000
embedding_dim = 100
maxlen = 512  # Mengubah panjang maksimum sequence menjadi 512
epochs = 10
batch_size = 8
num_folds = 5

# Seed
np.random.seed(42)
tf.random.set_seed(42)

# texts = df['text'].tolist()
label_mapping = {'positive': 2, 'negative': 0, 'neutral': 1}
# labels = df['label'].map(label_mapping).tolist()

texts_train = df['text'].tolist()
labels_train = df['label'].map(label_mapping).tolist()

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post')

labels_train = np.array(labels_train)

# K-fold cross-validation on training data
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

lstm_best_accuracy = 0.0
lstm_best_model = None
fold_no = 1
lstm_time_metrics = {}
lstm_results_accuracy = {}
lstm_results_precision = {}
lstm_results_recall = {}
lstm_results_f1 = {}
lstm_fold_metrics = {}

for train_index, val_index in kf.split(padded_sequences):
    print(f"Training fold {fold_no}...")
    start_time = time.time()

    X_train_fold, X_val_fold = padded_sequences[train_index], padded_sequences[val_index]
    y_train_fold, y_val_fold = labels_train[train_index], labels_train[val_index]

    # Build LSTM model
    model_lstm = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128, input_length=maxlen),
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.6),
        Bidirectional(LSTM(128)),
        Dropout(0.6),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dropout(0.6),
        Dense(3, activation='softmax')  # 3 label dengan aktivasi softma
    ])

    model_lstm.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model_lstm.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, validation_data=(X_val_fold, y_val_fold), verbose=2)

    # Evaluate the model on validation set
    val_loss, val_accuracy = model_lstm.evaluate(X_val_fold, y_val_fold, verbose=2)

    # Predict on validation set
    y_val_pred = model_lstm.predict(X_val_fold)
    y_val_pred_classes = np.argmax(y_val_pred, axis=1)

    # Compute evaluation metrics
    lstm_fold_accuracy = accuracy_score(y_val_fold, y_val_pred_classes)
    lstm_fold_precision = precision_score(y_val_fold, y_val_pred_classes, average='weighted')
    lstm_fold_recall = recall_score(y_val_fold, y_val_pred_classes, average='weighted')
    lstm_fold_f1 = f1_score(y_val_fold, y_val_pred_classes, average='weighted')

    end_time = time.time()
    lstm_fold_time = end_time - start_time
    lstm_time_metrics[fold_no] = lstm_fold_time

    print(f"Fold {fold_no} Validation Accuracy: {lstm_fold_accuracy}")
    print(f"Fold {fold_no} Validation Precision: {lstm_fold_precision}")
    print(f"Fold {fold_no} Validation Recall: {lstm_fold_recall}")
    print(f"Fold {fold_no} Validation F1 Score: {lstm_fold_f1}")

    lstm_results_accuracy[fold_no] = lstm_fold_accuracy
    lstm_results_precision[fold_no] = lstm_fold_precision
    lstm_results_recall[fold_no] = lstm_fold_recall
    lstm_results_f1[fold_no] = lstm_fold_f1

    lstm_fold_metrics[fold_no] = {
        'train_loss': history.history['loss'],
        'train_accuracy': history.history['accuracy'],
        'val_loss': history.history['val_loss'],
        'val_accuracy': history.history['val_accuracy']
    }

    if lstm_fold_accuracy > lstm_best_accuracy:
        lstm_best_accuracy = lstm_fold_accuracy
        lstm_best_model = model_lstm
        lstm_best_fold = fold_no

    fold_no += 1

if lstm_best_model:
    model_path = f'ms_lstm2_best_model_fold-{lstm_best_fold}-{lstm_best_accuracy}.h5'
    lstm_best_model.save(model_path)
    print(f"Best model saved to {model_path}")

# Print best model and its fold
print(f"Best fold: {lstm_best_fold} with validation accuracy: {lstm_best_accuracy}")
print(f"Time metrics: {lstm_time_metrics}")
print(f"Results accuracy: {lstm_results_accuracy}")
print(f"Results precision: {lstm_results_precision}")
print(f"Results recall: {lstm_results_recall}")
print(f"Results F1 score: {lstm_results_f1}")

In [ ]:
import joblib
from sklearn.model_selection import KFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define pipeline
PipeL = Pipeline(steps=[('tfidf', TfidfVectorizer()), ('svc', LinearSVC())])

# K-Fold Cross Validation
k = 5  # Number of folds
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

# Define scoring metrics with appropriate average for multiclass
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}

# Perform cross-validation and collect scores
scores = cross_validate(PipeL, X_train, y_train, cv=kfold, scoring=scoring, return_estimator=True)

# Find the best model based on accuracy
best_index = np.argmax(scores['test_accuracy'])
best_model = scores['estimator'][best_index]
best_accuracy = scores['test_accuracy'][best_index]

# Save the best model to a file with fold number and accuracy in the filename
joblib_file = f"best_model_fold{best_index+1}_acc{best_accuracy:.4f}.pkl"
joblib.dump(best_model, joblib_file)

# Print results
for metric in scoring:
    print(f'{metric.capitalize()} setiap fold: {scores[f"test_{metric}"]}')
    print(f'Rata-rata {metric}: {scores[f"test_{metric}"].mean():.4f}')

print(f"Best model pada fold {best_index+1} dengana akurasi {best_accuracy:.4f}")
